Thanks to RareSkill's awesome ZK book I've acquired enough knowledge to implement Groth16 in python.<br>Though I don't expect this to be easy, I'll leverage my giga brain and exceptional resilience to apply what I've learned to this complex and interesting project.

### Quick introduction to Groth16
Groth16 is a magic algorithm to validate that a statement is true without revealing any information about the inputs.<br>
For example, imagine that you want to prove to someone that you know a secret number that satisfies a specific equation without revealing the number itself; <br>the Groth16 algorithm could do just that.
It is known for its efficiency and small proof size, though it comes with one compromise: The need of a trusted setup.

### Polynomial Secret Proof PoC 
To dive into the practical implementation of Groth16, I'll start with a foundational example that showcases its core strengths.<br> 
I'm going to implement a system that proves knowledge of a secret value that satisfies a polynomial equation:<br> something like proving **I know a secret {x,y,z} such that $5x² + xy + 4z³ = 1059$**, without ever revealing what ${x,y,z}$ actually is. <BR><BR> The prover (likely the user) must provide the correct ${x,y,x}$ values which in our case are: $x = 13$,  $y = 14$,  $z = 2$ though they will stay private and the verifier will not have access to these raw values.

#### Steps and required setup overview (ordered):
- Define the R1CS constraint system based on our polynomial equation
- Convert R1CS -> QAP
- Implement the trusted setup phase to generate proving and verification keys
- Build the prover that creates ZK proofs using the secret witness
- Build the verifier that validates proofs without learning the secret
- Test the complete workflow with our polynomial example

---

### Defining the R1CS constraints

Lets say we are proving our earlier  equation $1059 = 5x² + xy + 4z³$, we want to prove that we know the solution ($x = 13$,  $y = 14$,  $z = 2$).<BR><BR>
We want to only have a single multiplication:<BR>

-  $v_1 = xx$ <BR>
-  $v_2 = 5v_1$
-  $v_3 = zz$
-  $v_4 = 4z * v_3$<BR><BR><BR> So we are left with a single mutliplication:<BR><BR>
$1059 = v_2 + xy + v_4$<BR><BR>
Simplified:<BR><BR>$1059 - v_2 - v_4 = xy$

<BR><BR>
we define the witness vector with all variables, including a constant $1$ as the first variable:<BR><BR>
let $out = 1059$<BR><BR>
let $w$ be the witness vector<BR><B>

$w = [1,out,x,y,z,v_1,v_2,v_3,v_4]$<BR>
$w = [1,1059,13,14,2,169,845,4,32]$
<BR><BR><B>then define $O=L*R$, with our 5 constraints<BR>

$L = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
5 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}$

$R = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}$

$O = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 0 & 0 & -1 & 0 & -1 \\
\end{bmatrix}$

Lets verify the circuit is correct with code

In [2]:
from py_ecc.optimized_bn128 import curve_order
import galois
import numpy as np

# define finite field on the bn128 EC (commonly used by Ethereum)
GF = galois.GF(curve_order) # = 21888242871839275222246405745257275088548364400416034343698204186575808495617

# witness vector
w = GF([1,1059,13,14,2,169,845,4,32])

# [1,out,x,y,z,v_1,v_2,v_3,v_4]
L = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
]))

R = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
]))

O = GF(np.array([
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, curve_order-1, 0, curve_order-1], # since we are working on a finite field i.e. -1 mod 17 = 16
]))

result = np.matmul(O, w) == np.multiply(np.matmul(L, w), np.matmul(R, w))

assert result.all(), "incorrect, should be equal"
print("Ow = Lw * Rw: " , result.all())



Ow = Lw * Rw:  True


#### As expected **it works**! If we changed even a single number it would obviously fail.

## R1CS -> QAP


*One Step Closer To **The Powers Of TAU!!** (exceptional naming sense).*<BR><BR> Now it is time to turn the R1CS to a QAP for the verifier, leveraging the Lagrange interpolations to represent the R1CS as polynomials for each column of all of the matrices,<BR> and the Schwartz-Zippel Lemma in order to achieve succintness. This helps groth16 be incredibly more efficient and massively reducing the proof size.


### First: transorm R1CS into polynomials

Here is the code for it, looping through each column:

In [3]:
# we have 5 constraints so interpolate 5 points
xs = GF(np.array([1,2,3,4,5]))

def interpolate_col(c):
    return galois.lagrange_poly(xs, c)

U_polys = []
V_polys = []
W_polys = []

# interpolate each column
for col_index in range(9):
    # extract column
    L_col = L[:, col_index]
    R_col = R[:, col_index]
    O_col = O[:, col_index]

    # get Lagrange'd
    U_poly = interpolate_col(L_col)
    V_poly = interpolate_col(R_col)
    W_poly = interpolate_col(O_col)

    U_polys.append(U_poly)
    V_polys.append(V_poly)
    W_polys.append(W_poly)

print("interplolated all columns\n")
print("Result:\n\n")
print(U_polys)
print(V_polys)
print(W_polys)


interplolated all columns

Result:


[Poly(18240202393199396018538671454381062573790303667013361953081836822146507079680x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415947x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301415887x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079770x + 21888242871839275222246405745257275088548364400416034343698204186575808495567, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)), Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)), Poly(20064222632519335620392538599819168831169334033714698148390020504361157787649x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495616x^3 + 12768141675239577212977070018066743801653212566909353367157285775502554955781x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247800x + 6, GF(218882428718392752222464057452572

### Completing the QAP equation
As of now we have computed this: <BR><BR>
$\underbrace{\sum_{i=1}^{5} a_i u_i(x)}_\text{U polys from L}  \underbrace{\sum_{i=1}^5 a_i v_i(x)}_\text{ V polys from R} = \underbrace{\sum_{i=1}^{5} a_i w_i(x)}_\text{ W polys from O}$
The verification is not succinct just yet, reason being that the verifier verifies all constraint points individually: 
- Check $U(1) \cdot V(1) = W(1)$
- $U(2) \cdot V(2) = W(2)$

And so on... we'd need 5 separate checks in our case.

#### Enabling succinctness
To enable succintness we need to introduce a set of two polynomial $h(x)$ and $t(x)$. $t(x)$ being $t(x) = (x - 1)(x - 2)...(x - 5)$<BR><BR>
knowing this we can compute $h(x)$ which formula is this: $h(x) = \frac{u(x)v(x) – w(x)}{t(x)}$. <BR><BR>
Now the verifier only needs ONE check, invoking the Powers of Tau...

## Trusted setup - Powers of Tau
Thanks to the work we just did we have laid all the groundwork are now able to evaluate the polynomial at a secret value $\tau$ (tau), though the QAP is not exactly complete yet.<BR>
Going back to our witness vector $[1,out,x,y,z,v_1,v_2,v_3,v_4]$, and knowing that the prover only wants to prove that he knows ${x,y,z}$ for which it satifies the equation $out = 1059$,<BR>
we want only want the first 2 inputs to be publics. So we define $l = 2$ so that inputs starting from $x$ to $v_4$ will not be known by the verifier.

### Computing t(x) with $\tau$:

In [5]:
import random

print("Trusted setup phase of the QAP implementation")
tau = GF(random.randint(1, 100000000))
# num of public inputs
l = 2
# num of private inputs
r = len(w) - l
deg = len(L) - 1
print("the degree of our polynomial is:", deg)
print("number of private and public inputs respectively:", r,"and", l)

# compute t(x) with the known roots
def compute_t(roots):
    # init t at = (x-1)
    t = galois.Poly([1, curve_order - 1], field=GF)
    for i in range(2, len(roots) + 1):
        t = t * galois.Poly([1, curve_order - i], field=GF)
    return t

T = compute_t(xs)

# check roots (for x in 1..=5 x should be root and !root if > 5 )
for i in range(1, 7):
    print(f"T({i}) = ", T(i))
    if i == 5 or i == 6:
        print("*"*10)

Ttau = T(tau)
print("\nTtau =", Ttau)

Trusted setup phase of the QAP implementation
the degree of our polynomial is: 4
number of private and public inputs respectively: 7 and 2
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
**********
T(6) =  120
**********

Ttau = 5592813083423918248966543682874233293440


time to compute the structured reference strings, here is the code following the RareSkills ZK book based on the computation instructions: 

$[\Omega_{n-1}, \Omega_{n-2},\dots,\Omega_2,\Omega_1,G_1] = [\tau^nG_1,\tau^{n-1}G_1,\dots,\tau G_1,G_1] \text{srs for } G_1$ <BR>
$[\Theta_{n-1}, \Theta_{n-2},\dots,\Theta_2,\Theta_2,G_2] = [\tau^nG_2,\tau^{n-1}G_2,\dots,\tau G_2,G_2] \text{srs for } G_2$ <BR>
$[\Upsilon_{n-2},\Upsilon_{n-3},\dots,\Upsilon_1,\Upsilon_0]=[\tau^{n-2}t(\tau)G_1,\tau^{n-3}t(\tau)G_1,\dots,\tau t(\tau)G_1,t(\tau)G_1]  \text{srs for }h(\tau)t(\tau)$

In [34]:
from py_ecc.optimized_bn128 import G1, G2, multiply, add, pairing

## compute the structured reference strings 
# srs for G1
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, L.shape[0])]
# srs for G2
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, L.shape[0])]
# srs for h(tau)t(tau)
ht = [multiply(G1, int(tau**i * Ttau)) for i in range(0, L.shape[0] - 1)]

Now compute $h(x)$, as we know:
$h(x) = \frac{u(x)v(x) – w(x)}{t(x)}$, we need to first compute these $u(x), v(x),  w(x)$ which are each the witness polynomial dot products.<BR>Note: remainder of the $h$ computation should be 0 

In [35]:


# init and compute u(x),v(x),w(x)
u_x = galois.Poly([0], field=GF)
v_x = galois.Poly([0], field=GF)  
w_x = galois.Poly([0], field=GF)

for i in range(len(w)):
    u_x += w[i] * U_polys[i]
    v_x += w[i] * V_polys[i]
    w_x += w[i] * W_polys[i]

# now h(x)
h_x = (u_x * v_x - w_x) // T
rem = (u_x * v_x - w_x) % T
assert rem == 0

print("h(x):", h_x)

h(x): 2812031202284906886191378515883747146792671815331226634433449843414253174800x^3 + 7220080113974760924004890784025837268791995201526122439761560408766325718895x^2 + 10032111316259667810196269299909584415584667016857349074195010252180578894130x + 10032111316259667810196269299909584415584667016857349074195010252180578893869


Very good, QAP is almost done, we only need to evaluate the polynomials at $\tau$:<BR>


$U(\tau) = \sum_{i=1}^m a_iu_i(\tau)$ <BR>
$V(\tau) = \sum_{i=1}^m a_iv_i(\tau)$ <BR>
$W(\tau) = \sum_{i=1}^m a_iw_i(\tau)$ <BR>

Making sure that the witness is mathematically correct, very simple thanks to homomorphism: $U(\tau) \cdot V(\tau) - W(\tau) = h(\tau) \cdot t(\tau)$


In [36]:
ut = u_x(tau)
vt = v_x(tau)

wt = w_x(tau) 
ht_check = h_x(tau)*Ttau

print(ut * vt - wt == ht_check)
assert ut * vt - wt == ht_check

True


In [38]:
def encrypt_poly(poly, points):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(points), "Poly degree mismatch"
    terms = [multiply(point, int(coeff)) for point, coeff in zip(points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])
    return evaluation


A_G1 = encrypt_poly(u_x, tau_G1)
B_G2 = encrypt_poly(v_x, tau_G2)
B_G1 = encrypt_poly(v_x, tau_G1)
C_G1 = encrypt_poly(w_x, tau_G1)
H_G1 = encrypt_poly(h_x, ht)
C_G1 = add(C_G1, H_G1)

print("A_G1:", A_G1)
print("B_G2:", B_G2)
print("B_G1:", B_G1)
print("C_G1:", C_G1)
print("H_G1:", H_G1)
print("C_G1:", C_G1)

assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Verification failed"

A_G1: (17063633786293039501920134707601563537472038617915562481089228803356217261440, 3734803377031825317906717909614831873225825684706779639302087822020715508121, 3945148716357888721442821430437559513914060781824693037946433305962200093464)
B_G2: ((594518841012214200051598444068784177205113269574535018051562332616770755194, 3128442221645249770986479067600472809591086197824709174724767557816723662741), (16198460312999507429354575304538914241292236746065102238021205192193616797994, 19533680978130598674145684453195246014672651781465495568981148328591129832561), (20456848875177545211890597586438817577140449593087295982920556614083424086009, 17358289810247385155428747984402866703959767010862624906095588240274337820627))
B_G1: (11646988243394549761788832137207356629720841677269863583864698901521347103175, 20203301105702671082590981918034722383518022414615921323858824704575196608005, 11086831257391847384516104414937604570194987998263491505841355728046553677411)
C_G1: (15251925115513042896034